<a href="https://colab.research.google.com/github/KimGriezmann/Bigcontest_2021/blob/master/working_space_SeongGwan/split_coding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt 

plt.rc('font', family='NanumBarunGothic')

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
def create_df():

  #연도별 데이터 경로
  HTS_dir = '/content/drive/MyDrive/빅콘테스트/01_제공데이터/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_HTS'
  Player_dir = '/content/drive/MyDrive/빅콘테스트/01_제공데이터/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_선수'
  Hitter_dir = '/content/drive/MyDrive/빅콘테스트/01_제공데이터/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_타자 기본'
  #단일 데이터 경로
  Team_dir = '/content/drive/MyDrive/빅콘테스트/01_제공데이터/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_팀.csv'
  Schedule_dir = '/content/drive/MyDrive/빅콘테스트/01_제공데이터/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_경기일정_2021.csv'
  val_dir = '/content/drive/MyDrive/빅콘테스트/02_평가데이터/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_평가데이터.xlsx'

  year = [2018, 2019, 2020, 2021]

  for i in year:
    globals()['HTS_{}'.format(i)]     = pd.read_csv(HTS_dir + '_' + str(i) + '.csv', encoding='CP949')
    globals()['Player_{}'.format(i)]  = pd.read_csv(Player_dir + '_' + str(i) + '.csv', encoding='CP949')
    globals()['Hitter_{}'.format(i)]  = pd.read_csv(Hitter_dir + '_' + str(i) + '.csv', encoding='CP949')

  global Team_df, Schedule_df, val_df
  Team_df = pd.read_csv(Team_dir, encoding='CP949')
  Schedule_df = pd.read_csv(Schedule_dir, encoding='CP949')
  val_df = pd.read_excel(val_dir)

######################

#연도별 데이터 하나로 통합
def concat_df():
  df_list = ['HTS', 'Player', 'Hitter']

#각 칼럼이 모두 같은지 검사 후 통합
  for i in df_list:
    if ((globals()['{}_2018'.format(i)].columns == globals()['{}_2019'.format(i)].columns).all()) and ((globals()['{}_2019'.format(i)].columns == globals()['{}_2020'.format(i)].columns).all()) and \
    ((globals()['{}_2020'.format(i)].columns == globals()['{}_2021'.format(i)].columns).all()):
      globals()['{}_df'.format(i)] = pd.concat( [ globals()['{}_2018'.format(i)], globals()['{}_2019'.format(i)], globals()['{}_2020'.format(i)], globals()['{}_2021'.format(i) ] ] )

#연도별 길이의 합이 통합된 데이터의 길이와 같은지 검사
  for i in df_list:
    if len(globals()['{}_2018'.format(i)]) + len(globals()['{}_2019'.format(i)]) + len(globals()['{}_2020'.format(i)]) + len(globals()['{}_2021'.format(i)]) == len(globals()['{}_df'.format(i)]):
      print('{} is complete!'.format(i))

######################

create_df()
concat_df()

HTS is complete!
Player is complete!
Hitter is complete!


In [31]:
df = pd.read_csv('/content/drive/MyDrive/빅콘테스트/ops_data_2021.csv', encoding='CP949')
df['day'] = df['day'].apply(lambda x: int(x.replace('-', '')))

df.head()

,name,day,vs,result,bat_order,position,start_member,ab,score,hit,2_hit,3_hit,homerun,tb,rbi,sb,cs,bb,hpp,ibb,so,gdp,sh,sf,avg,obp,slg,ops,pitch,avLI,RE24,WPA
0,강백호,404,한화,W 3:2,4,1B,1,3,0,2,0,0,0,2,0,2,0,1,0,0,0,0,0,0,0.667,NaN,0.667,1.417,16,1.14,0.89,0.122
1,강백호,406,LG,L 2:3,4,1B,1,4,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0.286,NaN,0.286,0.661,14,2.07,-1.00,-0.183
2,강백호,407,LG,W 7:3,4,1B,1,4,0,3,0,0,0,3,1,0,0,1,0,0,0,0,0,0,0.455,NaN,0.455,0.993,17,1.43,1.75,0.330
3,강백호,408,LG,L 3:7,4,1B,1,4,0,2,0,0,0,2,0,0,0,0,0,0,1,1,0,0,0.467,NaN,0.467,0.996,17,0.89,-0.76,-0.102
4,강백호,409,@삼성,L 5:7,4,1B,1,5,0,3,0,1,0,5,3,0,0,0,0,0,1,1,0,0,0.500,NaN,0.600,1.145,21,0.94,1.88,-0.122


In [128]:
hitter_list = list(set(df['name']))

def last_split(start_date, train_num, pred_num):
    
    later_df = df[ df['day']  >= int(start_date) ]
    split_result = pd.DataFrame()
    
    for player in hitter_list:
        pl_data = later_df[later_df['name'] == player]

        if len(pl_data) < train_num:
            print('{}은/는 경기 수가 {}보다 작기 때문에 삭제'.format(player, train_num))
            pass

        else:
            pl_data = pl_data.sort_values('day').reset_index(drop=True)

            tr_data = pl_data[:train_num]
            ts_data = pl_data[train_num:train_num+pred_num]

            sum_data = tr_data.iloc[:, 7:23].sum()
            sum_data['name'] = player
            sum_data['last_date'] = tr_data['day'].iloc[-1]

            sum_data['label'] = ts_data['ops'].mean()

            sum_result = sum_data.to_frame().transpose()

            split_result = pd.concat([split_result, sum_result])
    
    split_result = split_result.reset_index(drop=True)
    
    return split_result



def random_split(seed, pred_num):
    
    split_result = pd.DataFrame()
    
    for player in hitter_list:
        pl_data = df[df['name'] == player].reset_index(drop=True)

        if len(pl_data) < pred_num*2:
            print('{}은/는 경기 수가 {}보다 작기 때문에 삭제'.format(player, pred_num*2))
            pass

        else:
            pl_data = pl_data.sort_values('day').reset_index(drop=True)

            tr_data = pl_data.sample(n=len(pl_data) - pred_num, random_state=seed)
            tr_idx = list(tr_data.index)

            ts_data = pl_data.drop(tr_idx)

            sum_data = tr_data.iloc[:, 7:23].sum()
            sum_data['name'] = player

            sum_data['label'] = ts_data['ops'].mean()

            sum_result = sum_data.to_frame().transpose()

            split_result = pd.concat([split_result, sum_result])
    
    split_result = split_result.reset_index(drop=True)
    
    return split_result

In [131]:
last_split(start_date = '0301', train_num = 30, pred_num = 10)

권혁경은/는 경기 수가 30보다 작기 때문에 삭제
한석현은/는 경기 수가 30보다 작기 때문에 삭제
황윤호은/는 경기 수가 30보다 작기 때문에 삭제
문대원은/는 경기 수가 30보다 작기 때문에 삭제
이재율은/는 경기 수가 30보다 작기 때문에 삭제
이현석은/는 경기 수가 30보다 작기 때문에 삭제
남태혁은/는 경기 수가 30보다 작기 때문에 삭제
김병휘은/는 경기 수가 30보다 작기 때문에 삭제
윤형준은/는 경기 수가 30보다 작기 때문에 삭제
백동훈은/는 경기 수가 30보다 작기 때문에 삭제
박승욱은/는 경기 수가 30보다 작기 때문에 삭제
유민상은/는 경기 수가 30보다 작기 때문에 삭제
이해창은/는 경기 수가 30보다 작기 때문에 삭제
이성우은/는 경기 수가 30보다 작기 때문에 삭제
오승환은/는 경기 수가 30보다 작기 때문에 삭제
강민국은/는 경기 수가 30보다 작기 때문에 삭제
민병헌은/는 경기 수가 30보다 작기 때문에 삭제
이태훈은/는 경기 수가 30보다 작기 때문에 삭제
김은성은/는 경기 수가 30보다 작기 때문에 삭제
안익훈은/는 경기 수가 30보다 작기 때문에 삭제
김성표은/는 경기 수가 30보다 작기 때문에 삭제
김윤수은/는 경기 수가 30보다 작기 때문에 삭제
나승엽은/는 경기 수가 30보다 작기 때문에 삭제
최민재은/는 경기 수가 30보다 작기 때문에 삭제
이창재은/는 경기 수가 30보다 작기 때문에 삭제
장현식은/는 경기 수가 30보다 작기 때문에 삭제
지석훈은/는 경기 수가 30보다 작기 때문에 삭제
최정용은/는 경기 수가 30보다 작기 때문에 삭제
오준혁은/는 경기 수가 30보다 작기 때문에 삭제
문찬종은/는 경기 수가 30보다 작기 때문에 삭제
국해성은/는 경기 수가 30보다 작기 때문에 삭제
강로한은/는 경기 수가 30보다 작기 때문에 삭제
이재익은/는 경기 수가 30보다 작기 때문에 삭제
유원상은/는 경기 수가 30보다 작기 때문에 삭제
김태연은/는 경기 수가 30보다 작기 때문에 삭제
김건형은/는 경기 수가 30보다 작기

,ab,score,hit,2_hit,3_hit,homerun,tb,rbi,sb,cs,bb,hpp,ibb,so,gdp,sh,name,last_date,label
0,125,20,31,5,1,9,65,30,0,1,11,5,1,33,1,0,나성범,509,0.8922
1,36,3,9,1,0,0,10,3,0,0,4,0,0,8,0,0,전민수,820,0.594444
2,89,13,19,3,0,1,25,4,6,1,12,1,0,26,1,2,노수광,617,0.541
3,57,11,13,4,0,2,23,10,1,0,5,1,0,18,0,0,문상철,514,0.707
4,78,9,20,7,0,2,33,13,0,0,20,0,0,19,2,0,문보경,613,0.8674
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152,60,10,12,2,0,2,20,4,1,0,11,5,0,22,1,2,김호령,629,0.81
153,104,15,26,3,0,1,32,14,7,4,16,0,2,17,0,2,박해민,508,0.7321
154,113,24,38,7,0,11,78,26,3,1,10,3,0,32,1,0,알테어,509,1.0689
155,37,9,10,1,1,0,13,5,4,3,10,0,0,9,0,0,조수행,526,0.7752


In [132]:
random_split(seed = 1004, pred_num = 30)

권혁경은/는 경기 수가 60보다 작기 때문에 삭제
한석현은/는 경기 수가 60보다 작기 때문에 삭제
전민수은/는 경기 수가 60보다 작기 때문에 삭제
황윤호은/는 경기 수가 60보다 작기 때문에 삭제
노수광은/는 경기 수가 60보다 작기 때문에 삭제
문대원은/는 경기 수가 60보다 작기 때문에 삭제
문상철은/는 경기 수가 60보다 작기 때문에 삭제
이재율은/는 경기 수가 60보다 작기 때문에 삭제
최용제은/는 경기 수가 60보다 작기 때문에 삭제
이현석은/는 경기 수가 60보다 작기 때문에 삭제
남태혁은/는 경기 수가 60보다 작기 때문에 삭제
김병휘은/는 경기 수가 60보다 작기 때문에 삭제
윤형준은/는 경기 수가 60보다 작기 때문에 삭제
류지혁은/는 경기 수가 60보다 작기 때문에 삭제
김재성은/는 경기 수가 60보다 작기 때문에 삭제
이홍구은/는 경기 수가 60보다 작기 때문에 삭제
백동훈은/는 경기 수가 60보다 작기 때문에 삭제
박승욱은/는 경기 수가 60보다 작기 때문에 삭제
유민상은/는 경기 수가 60보다 작기 때문에 삭제
이해창은/는 경기 수가 60보다 작기 때문에 삭제
이성우은/는 경기 수가 60보다 작기 때문에 삭제
오승환은/는 경기 수가 60보다 작기 때문에 삭제
강민국은/는 경기 수가 60보다 작기 때문에 삭제
신용수은/는 경기 수가 60보다 작기 때문에 삭제
민병헌은/는 경기 수가 60보다 작기 때문에 삭제
최인호은/는 경기 수가 60보다 작기 때문에 삭제
이태훈은/는 경기 수가 60보다 작기 때문에 삭제
김은성은/는 경기 수가 60보다 작기 때문에 삭제
안익훈은/는 경기 수가 60보다 작기 때문에 삭제
김성표은/는 경기 수가 60보다 작기 때문에 삭제
변상권은/는 경기 수가 60보다 작기 때문에 삭제
김윤수은/는 경기 수가 60보다 작기 때문에 삭제
황대인은/는 경기 수가 60보다 작기 때문에 삭제
김재호은/는 경기 수가 60보다 작기 때문에 삭제
라모스은/는 경기 수가 60보다 작기 때문에 삭제
나승엽은/는 경기 수가 60보다 작기

,ab,score,hit,2_hit,3_hit,homerun,tb,rbi,sb,cs,bb,hpp,ibb,so,gdp,sh,name,label
0,257,45,67,15,1,16,132,52,1,0,21,7,2,65,4,0,나성범,0.950367
1,104,13,24,4,1,2,36,14,0,0,18,0,0,16,3,0,문보경,0.859867
2,98,10,23,1,1,0,26,12,2,1,6,0,1,8,4,1,이지영,0.5402
3,172,25,44,8,0,8,76,33,0,1,31,1,1,43,9,1,한동희,0.784767
4,29,22,8,0,0,0,8,4,5,0,5,0,0,13,0,1,송민섭,0.465867
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,206,22,53,7,1,5,77,27,0,0,31,2,1,25,5,0,터커,0.641867
98,82,10,18,7,0,0,25,5,0,0,21,0,1,16,1,1,김민식,0.7755
99,239,45,62,10,1,5,89,35,27,7,45,1,2,43,0,5,박해민,0.766
100,216,32,52,7,0,12,95,30,7,1,28,3,1,68,2,0,알테어,0.972067
